# Image classification transfer learning demo

1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
3. [Fine-tuning the Image classification model](#Fine-tuning-the-Image-classification-model)
4. [Deploy The Model](#Deploy-the-model)
  1. [Create model](#Create-model)
  2. [Batch transform](#Batch-transform)
  3. [Realtime inference](#Realtime-inference)
    1. [Create endpoint configuration](#Create-endpoint-configuration) 
    2. [Create endpoint](#Create-endpoint) 
    3. [Perform inference](#Perform-inference) 
    4. [Clean up](#Clean-up)


## Introduction

Welcome to our end-to-end example of distributed image classification algorithm in transfer learning mode. In this demo, we will use the Amazon sagemaker image classification algorithm in transfer learning mode to fine-tune a pre-trained model (trained on imagenet data) to learn to classify a new dataset. In particular, the pre-trained model will be fine-tuned using [caltech-256 dataset](http://www.vision.caltech.edu/Image_Datasets/Caltech256/). 

To get started, we need to set up the environment with a few prerequisite steps, for permissions, configurations, and so on.

## Prequisites and Preprocessing

### Permissions and environment variables

Here we set up the linkage and authentication to AWS services. There are three parts to this:

* The roles used to give learning and hosting access to your data. This will automatically be obtained from the role used to start the notebook
* The S3 bucket that you want to use for training and model data
* The Amazon sagemaker image classification docker image which need not be changed

In [38]:
%%time
import boto3
import re
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket='objectdetectchicagoteam2' # customize to your bucket

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

print(training_image)

811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1
CPU times: user 69.3 ms, sys: 0 ns, total: 69.3 ms
Wall time: 90.1 ms


## Fine-tuning the Image classification model

The caltech 256 dataset consist of images from 257 categories (the last one being a clutter category) and has 30k images with a minimum of 80 images and a maximum of about 800 images per category. 

The image classification algorithm can take two types of input formats. The first is a [recordio format](https://mxnet.incubator.apache.org/tutorials/basic/record_io.html) and the other is a [lst format](https://mxnet.incubator.apache.org/how_to/recordio.html?highlight=im2rec). Files for both these formats are available at http://data.dmlc.ml/mxnet/data/caltech-256/. In this example, we will use the recordio format for training and use the training/validation split [specified here](http://data.dmlc.ml/mxnet/data/caltech-256/).

In [63]:
import os 
import urllib.request
import boto3

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

        
def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)


# caltech-256
s3_train_key = "image-classification-transfer-learning/train"
s3_validation_key = "image-classification-transfer-learning/validation"
s3_train = 's3://{}/{}/'.format(bucket, s3_train_key)
s3_validation = 's3://{}/{}/'.format(bucket, s3_validation_key)

#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-train.rec')
#upload_to_s3(s3_train_key, 'caltech-256-60-train.rec')
#download('http://data.mxnet.io/data/caltech-256/caltech-256-60-val.rec')
#upload_to_s3(s3_validation_key, 'caltech-256-60-val.rec')

# Our tiny image set
s3_train_key = 'tiny3/train'
s3_train = 's3://{}/{}/'.format(bucket, s3_train_key)
s3_validation_key = 'tiny3/validation'
s3_validation = 's3://{}/{}/'.format(bucket, s3_validation_key)

Once we have the data available in the correct format for training, the next step is to actually train the model using the data. Before training the model, we need to setup the training parameters. The next section will explain the parameters in detail.

## Training parameters
There are two kinds of parameters that need to be set for training. The first one are the parameters for the training job. These include:

* **Input specification**: These are the training and validation channels that specify the path where training data is present. These are specified in the "InputDataConfig" section. The main parameters that need to be set is the "ContentType" which can be set to "application/x-recordio" or "application/x-image" based on the input data format and the S3Uri which specifies the bucket and the folder where the data is present. 
* **Output specification**: This is specified in the "OutputDataConfig" section. We just need to specify the path where the output can be stored after training
* **Resource config**: This section specifies the type of instance on which to run the training and the number of hosts used for training. If "InstanceCount" is more than 1, then training can be run in a distributed manner. 

Apart from the above set of parameters, there are hyperparameters that are specific to the algorithm. These are:

* **num_layers**: The number of layers (depth) for the network. We use 18 in this samples but other values such as 50, 152 can be used.
* **num_training_samples**: This is the total number of training samples. It is set to 15420 for caltech dataset with the current split
* **num_classes**: This is the number of output classes for the new dataset. Imagenet was trained with 1000 output classes but the number of output classes can be changed for fine-tuning. For caltech, we use 257 because it has 256 object categories + 1 clutter class
* **epochs**: Number of training epochs
* **learning_rate**: Learning rate for training
* **mini_batch_size**: The number of training samples used for each mini batch. In distributed training, the number of training samples used per batch will be N * mini_batch_size where N is the number of hosts on which training is run

After setting training parameters, we kick off training, and poll for status until training is completed, which in this example, takes between 10 to 12 minutes per epoch on a p2.xlarge machine. The network typically converges after 10 epochs. However, to save the training time, we set the epochs to 2 but please keep in mind that it may not be  sufficient to generate a good model. 

In [67]:
# The algorithm supports multiple network depth (number of layers). They are 18, 34, 50, 101, 152 and 200
# For this training, we will use 18 layers
num_layers = 18
# we need to specify the input image shape for the training data
image_shape = "3,224,224"
# we also need to specify the number of training samples in the training set
# for caltech it is 15420
num_training_samples = 10
# specify the number of output classes
num_classes = 2
# batch size for training
mini_batch_size =  5
# number of epochs
epochs = 2
# learning rate
learning_rate = 0.01
top_k=1
# Since we are using transfer learning, we set use_pretrained_model to 1 so that weights can be 
# initialized with pre-trained weights
use_pretrained_model = 1

# Training
Run the training using Amazon sagemaker CreateTrainingJob API

In [68]:
%%time
import time
import boto3
from time import gmtime, strftime


s3 = boto3.client('s3')
# create unique job name 
job_name_prefix = 'Scientist-36-imageclassification'
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = job_name_prefix + timestamp
training_params = \
{
    # specify the training docker image
    "AlgorithmSpecification": {
        "TrainingImage": training_image,
        "TrainingInputMode": "File"
    },
    "RoleArn": role,
    "OutputDataConfig": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, job_name_prefix)
    },
    "ResourceConfig": {
        "InstanceCount": 1,
        "InstanceType": "ml.p2.xlarge",
        "VolumeSizeInGB": 50
    },
    "TrainingJobName": job_name,
    "HyperParameters": {
        "image_shape": image_shape,
        "num_layers": str(num_layers),
        "num_training_samples": str(num_training_samples),
        "num_classes": str(num_classes),
        "mini_batch_size": str(mini_batch_size),
        "epochs": str(epochs),
        "learning_rate": str(learning_rate),
        "use_pretrained_model": str(use_pretrained_model)
    },
    "StoppingCondition": {
        "MaxRuntimeInSeconds": 360000
    },
#Training data should be inside a subdirectory called "train"
#Validation data should be inside a subdirectory called "validation"
#The algorithm currently only supports fullyreplicated model (where data is copied onto each machine)
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_train,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": s3_validation,
                    "S3DataDistributionType": "FullyReplicated"
                }
            },
            "ContentType": "application/x-recordio",
            "CompressionType": "None"
        }
    ]
    
}
print('Training job name: {}'.format(job_name))
print('\nInput Data Location: {}'.format(training_params['InputDataConfig'][0]['DataSource']['S3DataSource']))

Training job name: Scientist-36-imageclassification-2019-02-13-21-13-49

Input Data Location: {'S3DataType': 'S3Prefix', 'S3Uri': 's3://objectdetectchicagoteam2/tiny3/train/', 'S3DataDistributionType': 'FullyReplicated'}
CPU times: user 5.09 ms, sys: 0 ns, total: 5.09 ms
Wall time: 4.95 ms


"InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "AugmentedManifestFile",
                    "S3Uri": s3_train,
                    "S3DataDistributionType": "FullyReplicated",
                    "AttributeNames": ['source-ref', 'class']
                }
            },
            "InputMode": "Pipe",
            "ContentType": "image/jpeg",
            "CompressionType": "None",
            "RecordWrapperType": "None"
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "AugmentedManifestFile",
                    "S3Uri": s3_validation,
                    "S3DataDistributionType": "FullyReplicated",
                    "AttributeNames": ['source-ref', 'class']
                }
            },
            "InputMode": "Pipe",
            "ContentType": "image/jpeg",
            "CompressionType": "None",
            "RecordWrapperType": "RecordIO"
        }
    ]

In [69]:
# create the Amazon SageMaker training job
sagemaker = boto3.client(service_name='sagemaker')
sagemaker.create_training_job(**training_params)

# confirm that the training job has started
status = sagemaker.describe_training_job(TrainingJobName=job_name)['TrainingJobStatus']
print('Training job current status: {}'.format(status))

try:
    # wait for the job to finish and report the ending status
    sagemaker.get_waiter('training_job_completed_or_stopped').wait(TrainingJobName=job_name)
    training_info = sagemaker.describe_training_job(TrainingJobName=job_name)
    status = training_info['TrainingJobStatus']
    print("Training job ended with status: " + status)
except:
    print('Training failed to start')
     # if exception is raised, that means it has failed
    message = sagemaker.describe_training_job(TrainingJobName=job_name)['FailureReason']
    print('Training failed with the following error: {}'.format(message))

Training job current status: InProgress
Training job ended with status: Completed


In [70]:
training_info = sagemaker.describe_training_job(TrainingJobName=job_name)
status = training_info['TrainingJobStatus']
print("Training job ended with status: " + status)

Training job ended with status: Completed


If you see the message,

> `Training job ended with status: Completed`

then that means training sucessfully completed and the output model was stored in the output path specified by `training_params['OutputDataConfig']`.

You can also view information about and the status of a training job using the AWS SageMaker console. Just click on the "Jobs" tab.

# Deploy The Model

***

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the topic mixture representing a given document.

Image-classification only supports encoded .jpg and .png image formats as inference input for now. The output is the probability values for all classes encoded in JSON format, or in JSON Lines format for batch transform.

This section involves several steps,

1. [Create Model](#CreateModel) - Create model for the training output
1. [Batch Transform](#BatchTransform) - Create a transform job to perform batch inference.
1. [Host the model for realtime inference](#HostTheModel) - Create an inference endpoint and perform realtime inference.

## Create Model

We now create a SageMaker Model from the training output. Using the model we can create an Endpoint Configuration.

In [71]:
%%time
import boto3
from time import gmtime, strftime

sage = boto3.Session().client(service_name='sagemaker') 

model_name=job_name_prefix
print(model_name)
info = sage.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

Scientist-36-imageclassification
s3://objectdetectchicagoteam2/Scientist-36-imageclassification/output/Scientist-36-imageclassification-2019-02-13-21-13-49/output/model.tar.gz
arn:aws:sagemaker:us-east-1:023375022819:model/scientist-36-imageclassification
CPU times: user 68.6 ms, sys: 0 ns, total: 68.6 ms
Wall time: 407 ms


### Batch transform

We now create a SageMaker Batch Transform job using the model created above to perform batch prediction.

In [72]:
!ls -l

total 707168
-rw-rw-r-- 1 ec2-user ec2-user 363622688 Feb 13 17:41 caltech-256-60-train.rec
-rw-rw-r-- 1 ec2-user ec2-user 360242580 Feb 13 17:41 caltech-256-60-val.rec
-rw-r--r-- 1 ec2-user ec2-user     29221 Feb  6 00:54 Image-classification-fulltraining-elastic-inference.ipynb
-rw-r--r-- 1 ec2-user ec2-user     17834 Feb  6 00:54 Image-classification-fulltraining-highlevel.ipynb
-rw-r--r-- 1 ec2-user ec2-user     21320 Feb  6 00:54 Image-classification-fulltraining-highlevel-neo.ipynb
-rw-r--r-- 1 ec2-user ec2-user     36415 Feb  6 00:54 Image-classification-fulltraining.ipynb
-rw-r--r-- 1 ec2-user ec2-user     20340 Feb  6 00:54 Image-classification-incremental-training-highlevel.ipynb
-rw-r--r-- 1 ec2-user ec2-user     22225 Feb  6 00:54 Image-classification-lst-format-highlevel.ipynb
-rw-r--r-- 1 ec2-user ec2-user     42299 Feb  6 00:54 Image-classification-lst-format.ipynb
-rw-r--r-- 1 ec2-user ec2-user     17762 Feb  6 00:54 Image-classification-transfer-learning-highlevel.ipyn

#### Download test data


In [73]:
# Download images under /008.bathtub
!wget -r -np -nH --cut-dirs=2 -P /tmp/ -R "index.html*" http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/


--2019-02-13 21:33:05--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 34.208.54.77
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|34.208.54.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32675 (32K) [text/html]
Saving to: ‘/tmp/images/008.bathtub/index.html’

images/008.bathtub/ 100%[===================>]  31.91K  --.-KB/s    in 0.08s   

2019-02-13 21:33:05 (398 KB/s) - ‘/tmp/images/008.bathtub/index.html’ saved [32675/32675]

Loading robots.txt; please ignore errors.
--2019-02-13 21:33:05--  http://www.vision.caltech.edu/robots.txt
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 1307 (1.3K) [text/plain]
Saving to: ‘/tmp/robots.txt’

robots.txt          100%[===================>]   1.28K  --.-KB/s    in 0s      

2019-02-13 21:33:05 (127 MB/s) - ‘/tmp/robots.txt’ saved [1307/1307]

Remov

HTTP request sent, awaiting response... 200 OK
Length: 3692 (3.6K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0008.jpg’

images/008.bathtub/ 100%[===================>]   3.61K  --.-KB/s    in 0s      

2019-02-13 21:33:07 (369 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0008.jpg’ saved [3692/3692]

--2019-02-13 21:33:07--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0009.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 6316 (6.2K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0009.jpg’

images/008.bathtub/ 100%[===================>]   6.17K  --.-KB/s    in 0s      

2019-02-13 21:33:07 (590 MB/s) - ‘/tmp/images/008.bathtub/008_0009.jpg’ saved [6316/6316]

--2019-02-13 21:33:07--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0009.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiti

HTTP request sent, awaiting response... 200 OK
Length: 18400 (18K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0017.jpg’

images/008.bathtub/ 100%[===================>]  17.97K  --.-KB/s    in 0s      

2019-02-13 21:33:09 (61.4 MB/s) - ‘/tmp/images/008.bathtub/008_0017.jpg’ saved [18400/18400]

--2019-02-13 21:33:09--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0017.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 2842 (2.8K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0017.jpg’

images/008.bathtub/ 100%[===================>]   2.78K  --.-KB/s    in 0s      

2019-02-13 21:33:09 (330 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0017.jpg’ saved [2842/2842]

--2019-02-13 21:33:09--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0018.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awa

HTTP request sent, awaiting response... 200 OK
Length: 3743 (3.7K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0025.jpg’

images/008.bathtub/ 100%[===================>]   3.66K  --.-KB/s    in 0s      

2019-02-13 21:33:11 (434 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0025.jpg’ saved [3743/3743]

--2019-02-13 21:33:11--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0026.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 7555 (7.4K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0026.jpg’

images/008.bathtub/ 100%[===================>]   7.38K  --.-KB/s    in 0s      

2019-02-13 21:33:11 (680 MB/s) - ‘/tmp/images/008.bathtub/008_0026.jpg’ saved [7555/7555]

--2019-02-13 21:33:11--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0026.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiti

HTTP request sent, awaiting response... 200 OK
Length: 13907 (14K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0034.jpg’

images/008.bathtub/ 100%[===================>]  13.58K  --.-KB/s    in 0s      

2019-02-13 21:33:13 (195 MB/s) - ‘/tmp/images/008.bathtub/008_0034.jpg’ saved [13907/13907]

--2019-02-13 21:33:13--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0034.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 3123 (3.0K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0034.jpg’

images/008.bathtub/ 100%[===================>]   3.05K  --.-KB/s    in 0s      

2019-02-13 21:33:13 (367 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0034.jpg’ saved [3123/3123]

--2019-02-13 21:33:13--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0035.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awai

...

HTTP request sent, awaiting response... 200 OK
Length: 3441 (3.4K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0195.jpg’

images/008.bathtub/ 100%[===================>]   3.36K  --.-KB/s    in 0s      

2019-02-13 21:33:52 (223 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0195.jpg’ saved [3441/3441]

--2019-02-13 21:33:52--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0196.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 9057 (8.8K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0196.jpg’

images/008.bathtub/ 100%[===================>]   8.84K  --.-KB/s    in 0s      

2019-02-13 21:33:53 (172 MB/s) - ‘/tmp/images/008.bathtub/008_0196.jpg’ saved [9057/9057]

--2019-02-13 21:33:53--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0196.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiti

HTTP request sent, awaiting response... 200 OK
Length: 24379 (24K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0204.jpg’

images/008.bathtub/ 100%[===================>]  23.81K  --.-KB/s    in 0.08s   

2019-02-13 21:33:55 (295 KB/s) - ‘/tmp/images/008.bathtub/008_0204.jpg’ saved [24379/24379]

--2019-02-13 21:33:55--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0204.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 4788 (4.7K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0204.jpg’

images/008.bathtub/ 100%[===================>]   4.68K  --.-KB/s    in 0s      

2019-02-13 21:33:55 (491 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0204.jpg’ saved [4788/4788]

--2019-02-13 21:33:55--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0205.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awai

HTTP request sent, awaiting response... 200 OK
Length: 4300 (4.2K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0212.jpg’

images/008.bathtub/ 100%[===================>]   4.20K  --.-KB/s    in 0s      

2019-02-13 21:33:57 (437 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0212.jpg’ saved [4300/4300]

--2019-02-13 21:33:57--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0213.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 9712 (9.5K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0213.jpg’

images/008.bathtub/ 100%[===================>]   9.48K  --.-KB/s    in 0s      

2019-02-13 21:33:57 (145 MB/s) - ‘/tmp/images/008.bathtub/008_0213.jpg’ saved [9712/9712]

--2019-02-13 21:33:57--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0213.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiti

HTTP request sent, awaiting response... 200 OK
Length: 9587 (9.4K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0221.jpg’

images/008.bathtub/ 100%[===================>]   9.36K  --.-KB/s    in 0s      

2019-02-13 21:33:59 (180 MB/s) - ‘/tmp/images/008.bathtub/008_0221.jpg’ saved [9587/9587]

--2019-02-13 21:33:59--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0221.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 5085 (5.0K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0221.jpg’

images/008.bathtub/ 100%[===================>]   4.97K  --.-KB/s    in 0s      

2019-02-13 21:33:59 (512 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0221.jpg’ saved [5085/5085]

--2019-02-13 21:33:59--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0222.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiti

HTTP request sent, awaiting response... 200 OK
Length: 6274 (6.1K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/thumbdir/008_0229.jpg’

images/008.bathtub/ 100%[===================>]   6.13K  --.-KB/s    in 0s      

2019-02-13 21:34:01 (551 MB/s) - ‘/tmp/images/008.bathtub/thumbdir/008_0229.jpg’ saved [6274/6274]

--2019-02-13 21:34:01--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0230.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awaiting response... 200 OK
Length: 21275 (21K) [image/jpeg]
Saving to: ‘/tmp/images/008.bathtub/008_0230.jpg’

images/008.bathtub/ 100%[===================>]  20.78K  --.-KB/s    in 0.001s  

2019-02-13 21:34:01 (24.1 MB/s) - ‘/tmp/images/008.bathtub/008_0230.jpg’ saved [21275/21275]

--2019-02-13 21:34:01--  http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/thumbdir/008_0230.jpg
Reusing existing connection to www.vision.caltech.edu:80.
HTTP request sent, awa

In [74]:
batch_input = 's3://{}/image-classification-transfer-learning/test/'.format(bucket)
test_images = '/tmp/images/008.bathtub'

!aws s3 cp $test_images $batch_input --recursive --quiet 

#### Create batch transform job

In [75]:
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
batch_job_name = "image-classification-model" + timestamp
request = \
{
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms": 16,
    "MaxPayloadInMB": 6,
    "BatchStrategy": "SingleRecord",
    "TransformOutput": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, batch_job_name)
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": batch_input
            }
        },
        "ContentType": "application/x-image",
        "SplitType": "None",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.p2.xlarge",
            "InstanceCount": 1
    }
}

print('Transform job name: {}'.format(batch_job_name))
print('\nInput Data Location: {}'.format(s3_validation))

Transform job name: image-classification-model-2019-02-13-21-34-40

Input Data Location: s3://objectdetectchicagoteam2/tiny3/validation/


In [76]:
sagemaker = boto3.client('sagemaker')
sagemaker.create_transform_job(**request)

print("Created Transform job with name: ", batch_job_name)

while(True):
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response['TransformJobStatus']
    if status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    time.sleep(30)  

Created Transform job with name:  image-classification-model-2019-02-13-21-34-40
Transform job ended with status: Completed


After the job completes, let's inspect the prediction results. The accuracy may not be quite good because we set the epochs to 2 during training which may not be sufficient to train a good model. 

In [77]:
from urllib.parse import urlparse
import json
import numpy as np

s3_client = boto3.client('s3')
#object_categories = ['ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'basketball-hoop', 'bat', 'bathtub', 'bear', 'beer-mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai-101', 'boom-box', 'bowling-ball', 'bowling-pin', 'boxing-glove', 'brain-101', 'breadmaker', 'buddha-101', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car-tire', 'cartman', 'cd', 'centipede', 'cereal-box', 'chandelier-101', 'chess-board', 'chimp', 'chopsticks', 'cockroach', 'coffee-mug', 'coffin', 'coin', 'comet', 'computer-keyboard', 'computer-monitor', 'computer-mouse', 'conch', 'cormorant', 'covered-wagon', 'cowboy-hat', 'crab-101', 'desk-globe', 'diamond-ring', 'dice', 'dog', 'dolphin-101', 'doorknob', 'drinking-straw', 'duck', 'dumb-bell', 'eiffel-tower', 'electric-guitar-101', 'elephant-101', 'elk', 'ewer-101', 'eyeglasses', 'fern', 'fighter-jet', 'fire-extinguisher', 'fire-hydrant', 'fire-truck', 'fireworks', 'flashlight', 'floppy-disk', 'football-helmet', 'french-horn', 'fried-egg', 'frisbee', 'frog', 'frying-pan', 'galaxy', 'gas-pump', 'giraffe', 'goat', 'golden-gate-bridge', 'goldfish', 'golf-ball', 'goose', 'gorilla', 'grand-piano-101', 'grapes', 'grasshopper', 'guitar-pick', 'hamburger', 'hammock', 'harmonica', 'harp', 'harpsichord', 'hawksbill-101', 'head-phones', 'helicopter-101', 'hibiscus', 'homer-simpson', 'horse', 'horseshoe-crab', 'hot-air-balloon', 'hot-dog', 'hot-tub', 'hourglass', 'house-fly', 'human-skeleton', 'hummingbird', 'ibis-101', 'ice-cream-cone', 'iguana', 'ipod', 'iris', 'jesus-christ', 'joy-stick', 'kangaroo-101', 'kayak', 'ketch-101', 'killer-whale', 'knife', 'ladder', 'laptop-101', 'lathe', 'leopards-101', 'license-plate', 'lightbulb', 'light-house', 'lightning', 'llama-101', 'mailbox', 'mandolin', 'mars', 'mattress', 'megaphone', 'menorah-101', 'microscope', 'microwave', 'minaret', 'minotaur', 'motorbikes-101', 'mountain-bike', 'mushroom', 'mussels', 'necktie', 'octopus', 'ostrich', 'owl', 'palm-pilot', 'palm-tree', 'paperclip', 'paper-shredder', 'pci-card', 'penguin', 'people', 'pez-dispenser', 'photocopier', 'picnic-table', 'playing-card', 'porcupine', 'pram', 'praying-mantis', 'pyramid', 'raccoon', 'radio-telescope', 'rainbow', 'refrigerator', 'revolver-101', 'rifle', 'rotary-phone', 'roulette-wheel', 'saddle', 'saturn', 'school-bus', 'scorpion-101', 'screwdriver', 'segway', 'self-propelled-lawn-mower', 'sextant', 'sheet-music', 'skateboard', 'skunk', 'skyscraper', 'smokestack', 'snail', 'snake', 'sneaker', 'snowmobile', 'soccer-ball', 'socks', 'soda-can', 'spaghetti', 'speed-boat', 'spider', 'spoon', 'stained-glass', 'starfish-101', 'steering-wheel', 'stirrups', 'sunflower-101', 'superman', 'sushi', 'swan', 'swiss-army-knife', 'sword', 'syringe', 'tambourine', 'teapot', 'teddy-bear', 'teepee', 'telephone-box', 'tennis-ball', 'tennis-court', 'tennis-racket', 'theodolite', 'toaster', 'tomato', 'tombstone', 'top-hat', 'touring-bike', 'tower-pisa', 'traffic-light', 'treadmill', 'triceratops', 'tricycle', 'trilobite-101', 'tripod', 't-shirt', 'tuning-fork', 'tweezer', 'umbrella-101', 'unicorn', 'vcr', 'video-projector', 'washing-machine', 'watch-101', 'waterfall', 'watermelon', 'welding-mask', 'wheelbarrow', 'windmill', 'wine-bottle', 'xylophone', 'yarmulke', 'yo-yo', 'zebra', 'airplanes-101', 'car-side-101', 'faces-easy-101', 'greyhound', 'tennis-shoes', 'toad', 'clutter']
object_categories = ['NO_SHIP', 'SHIP']

def list_objects(s3_client, bucket, prefix):
    response = s3_client.list_objects(Bucket=bucket, Prefix=prefix)
    objects = [content['Key'] for content in response['Contents']]
    return objects

def get_label(s3_client, bucket, prefix):
    filename = prefix.split('/')[-1]
    s3_client.download_file(bucket, prefix, filename)
    with open(filename) as f:
        data = json.load(f)
        index = np.argmax(data['prediction'])
        probability = data['prediction'][index]
    print("Result: label - " + object_categories[index] + ", probability - " + str(probability))
    return object_categories[index], probability

inputs = list_objects(s3_client, bucket, urlparse(batch_input).path.lstrip('/'))
print("Sample inputs: " + str(inputs[:2]))

outputs = list_objects(s3_client, bucket, batch_job_name + "/output")
print("Sample output: " + str(outputs[:2]))

# Check prediction result of the first 2 images
[get_label(s3_client, bucket, prefix) for prefix in outputs[0:10]]

Sample inputs: ['image-classification-transfer-learning/test/008_0001.jpg', 'image-classification-transfer-learning/test/008_0002.jpg']
Sample output: ['image-classification-model-2019-02-13-21-34-40/output/008_0001.jpg.out', 'image-classification-model-2019-02-13-21-34-40/output/008_0002.jpg.out']
Result: label - NO_SHIP, probability - 0.9831468462944031
Result: label - NO_SHIP, probability - 0.9092593193054199
Result: label - NO_SHIP, probability - 0.8947139382362366
Result: label - NO_SHIP, probability - 0.996058464050293
Result: label - NO_SHIP, probability - 0.9294375777244568
Result: label - NO_SHIP, probability - 0.9908691644668579
Result: label - NO_SHIP, probability - 0.9968734383583069
Result: label - NO_SHIP, probability - 0.9763967394828796
Result: label - NO_SHIP, probability - 0.9611251354217529
Result: label - NO_SHIP, probability - 0.9454469084739685


[('NO_SHIP', 0.9831468462944031),
 ('NO_SHIP', 0.9092593193054199),
 ('NO_SHIP', 0.8947139382362366),
 ('NO_SHIP', 0.996058464050293),
 ('NO_SHIP', 0.9294375777244568),
 ('NO_SHIP', 0.9908691644668579),
 ('NO_SHIP', 0.9968734383583069),
 ('NO_SHIP', 0.9763967394828796),
 ('NO_SHIP', 0.9611251354217529),
 ('NO_SHIP', 0.9454469084739685)]

#### Evaluate real no-ship validation images

In [85]:
batch_input = 's3://{}/tiny3/test/NO_SHIP'.format(bucket)

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
batch_job_name = "ships-model-no-ship-validation" + timestamp
request = \
{
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms": 16,
    "MaxPayloadInMB": 6,
    "BatchStrategy": "SingleRecord",
    "TransformOutput": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, batch_job_name)
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": batch_input
            }
        },
        "ContentType": "application/x-image",
        "SplitType": "None",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.p2.xlarge",
            "InstanceCount": 1
    }
}

print('Transform job name: {}'.format(batch_job_name))
print('\nInput Data Location: {}'.format(batch_input))

Transform job name: ships-model-no-ship-validation-2019-02-13-21-57-21

Input Data Location: s3://objectdetectchicagoteam2/tiny3/test/NO_SHIP


In [82]:
sagemaker = boto3.client('sagemaker')
sagemaker.create_transform_job(**request)

print("Created Transform job with name: ", batch_job_name)

while(True):
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response['TransformJobStatus']
    if status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    time.sleep(30)  

Created Transform job with name:  ships-model-no-ship-validation-2019-02-13-21-51-48
Transform job ended with status: Completed


In [83]:
from urllib.parse import urlparse
import json
import numpy as np

s3_client = boto3.client('s3')
#object_categories = ['ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'basketball-hoop', 'bat', 'bathtub', 'bear', 'beer-mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai-101', 'boom-box', 'bowling-ball', 'bowling-pin', 'boxing-glove', 'brain-101', 'breadmaker', 'buddha-101', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car-tire', 'cartman', 'cd', 'centipede', 'cereal-box', 'chandelier-101', 'chess-board', 'chimp', 'chopsticks', 'cockroach', 'coffee-mug', 'coffin', 'coin', 'comet', 'computer-keyboard', 'computer-monitor', 'computer-mouse', 'conch', 'cormorant', 'covered-wagon', 'cowboy-hat', 'crab-101', 'desk-globe', 'diamond-ring', 'dice', 'dog', 'dolphin-101', 'doorknob', 'drinking-straw', 'duck', 'dumb-bell', 'eiffel-tower', 'electric-guitar-101', 'elephant-101', 'elk', 'ewer-101', 'eyeglasses', 'fern', 'fighter-jet', 'fire-extinguisher', 'fire-hydrant', 'fire-truck', 'fireworks', 'flashlight', 'floppy-disk', 'football-helmet', 'french-horn', 'fried-egg', 'frisbee', 'frog', 'frying-pan', 'galaxy', 'gas-pump', 'giraffe', 'goat', 'golden-gate-bridge', 'goldfish', 'golf-ball', 'goose', 'gorilla', 'grand-piano-101', 'grapes', 'grasshopper', 'guitar-pick', 'hamburger', 'hammock', 'harmonica', 'harp', 'harpsichord', 'hawksbill-101', 'head-phones', 'helicopter-101', 'hibiscus', 'homer-simpson', 'horse', 'horseshoe-crab', 'hot-air-balloon', 'hot-dog', 'hot-tub', 'hourglass', 'house-fly', 'human-skeleton', 'hummingbird', 'ibis-101', 'ice-cream-cone', 'iguana', 'ipod', 'iris', 'jesus-christ', 'joy-stick', 'kangaroo-101', 'kayak', 'ketch-101', 'killer-whale', 'knife', 'ladder', 'laptop-101', 'lathe', 'leopards-101', 'license-plate', 'lightbulb', 'light-house', 'lightning', 'llama-101', 'mailbox', 'mandolin', 'mars', 'mattress', 'megaphone', 'menorah-101', 'microscope', 'microwave', 'minaret', 'minotaur', 'motorbikes-101', 'mountain-bike', 'mushroom', 'mussels', 'necktie', 'octopus', 'ostrich', 'owl', 'palm-pilot', 'palm-tree', 'paperclip', 'paper-shredder', 'pci-card', 'penguin', 'people', 'pez-dispenser', 'photocopier', 'picnic-table', 'playing-card', 'porcupine', 'pram', 'praying-mantis', 'pyramid', 'raccoon', 'radio-telescope', 'rainbow', 'refrigerator', 'revolver-101', 'rifle', 'rotary-phone', 'roulette-wheel', 'saddle', 'saturn', 'school-bus', 'scorpion-101', 'screwdriver', 'segway', 'self-propelled-lawn-mower', 'sextant', 'sheet-music', 'skateboard', 'skunk', 'skyscraper', 'smokestack', 'snail', 'snake', 'sneaker', 'snowmobile', 'soccer-ball', 'socks', 'soda-can', 'spaghetti', 'speed-boat', 'spider', 'spoon', 'stained-glass', 'starfish-101', 'steering-wheel', 'stirrups', 'sunflower-101', 'superman', 'sushi', 'swan', 'swiss-army-knife', 'sword', 'syringe', 'tambourine', 'teapot', 'teddy-bear', 'teepee', 'telephone-box', 'tennis-ball', 'tennis-court', 'tennis-racket', 'theodolite', 'toaster', 'tomato', 'tombstone', 'top-hat', 'touring-bike', 'tower-pisa', 'traffic-light', 'treadmill', 'triceratops', 'tricycle', 'trilobite-101', 'tripod', 't-shirt', 'tuning-fork', 'tweezer', 'umbrella-101', 'unicorn', 'vcr', 'video-projector', 'washing-machine', 'watch-101', 'waterfall', 'watermelon', 'welding-mask', 'wheelbarrow', 'windmill', 'wine-bottle', 'xylophone', 'yarmulke', 'yo-yo', 'zebra', 'airplanes-101', 'car-side-101', 'faces-easy-101', 'greyhound', 'tennis-shoes', 'toad', 'clutter']
object_categories = ['NO_SHIP', 'SHIP']

def list_objects(s3_client, bucket, prefix):
    response = s3_client.list_objects(Bucket=bucket, Prefix=prefix)
    objects = [content['Key'] for content in response['Contents']]
    return objects

def get_label(s3_client, bucket, prefix):
    filename = prefix.split('/')[-1]
    s3_client.download_file(bucket, prefix, filename)
    with open(filename) as f:
        data = json.load(f)
        index = np.argmax(data['prediction'])
        probability = data['prediction'][index]
    print("Result: label - " + object_categories[index] + ", probability - " + str(probability))
    return object_categories[index], probability

inputs = list_objects(s3_client, bucket, urlparse(batch_input).path.lstrip('/'))
print("Sample inputs: " + str(inputs[:2]))

outputs = list_objects(s3_client, bucket, batch_job_name + "/output")
print("Sample output: " + str(outputs[:2]))

# Check prediction result of the first 2 images
[get_label(s3_client, bucket, prefix) for prefix in outputs[0:10]]

Sample inputs: ['tiny3/test/NO_SHIP/2f3d7c434.jpg', 'tiny3/test/NO_SHIP/31d6fdf95.jpg']
Sample output: ['ships-model-no-ship-validation-2019-02-13-21-51-48/output/2f3d7c434.jpg.out', 'ships-model-no-ship-validation-2019-02-13-21-51-48/output/31d6fdf95.jpg.out']
Result: label - NO_SHIP, probability - 0.9732958078384399
Result: label - NO_SHIP, probability - 0.999259889125824
Result: label - NO_SHIP, probability - 0.9827027320861816
Result: label - NO_SHIP, probability - 0.9720324277877808
Result: label - NO_SHIP, probability - 0.9538697600364685
Result: label - NO_SHIP, probability - 0.8932860493659973
Result: label - NO_SHIP, probability - 0.9279369115829468


[('NO_SHIP', 0.9732958078384399),
 ('NO_SHIP', 0.999259889125824),
 ('NO_SHIP', 0.9827027320861816),
 ('NO_SHIP', 0.9720324277877808),
 ('NO_SHIP', 0.9538697600364685),
 ('NO_SHIP', 0.8932860493659973),
 ('NO_SHIP', 0.9279369115829468)]

#### Evaluate real no-ship validation images

In [86]:
batch_input = 's3://{}/tiny3/test/SHIP'.format(bucket)

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
batch_job_name = "ships-model-ship-validation" + timestamp
request = \
{
    "TransformJobName": batch_job_name,
    "ModelName": model_name,
    "MaxConcurrentTransforms": 16,
    "MaxPayloadInMB": 6,
    "BatchStrategy": "SingleRecord",
    "TransformOutput": {
        "S3OutputPath": 's3://{}/{}/output'.format(bucket, batch_job_name)
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": batch_input
            }
        },
        "ContentType": "application/x-image",
        "SplitType": "None",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.p2.xlarge",
            "InstanceCount": 1
    }
}

print('Transform job name: {}'.format(batch_job_name))
print('\nInput Data Location: {}'.format(batch_input))

Transform job name: ships-model-ship-validation-2019-02-13-21-57-21

Input Data Location: s3://objectdetectchicagoteam2/tiny3/test/SHIP


In [87]:
sagemaker = boto3.client('sagemaker')
sagemaker.create_transform_job(**request)

print("Created Transform job with name: ", batch_job_name)

while(True):
    response = sagemaker.describe_transform_job(TransformJobName=batch_job_name)
    status = response['TransformJobStatus']
    if status == 'Completed':
        print("Transform job ended with status: " + status)
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('Transform failed with the following error: {}'.format(message))
        raise Exception('Transform job failed') 
    time.sleep(30)  

Created Transform job with name:  ships-model-ship-validation-2019-02-13-21-57-21
Transform job ended with status: Completed


In [88]:
from urllib.parse import urlparse
import json
import numpy as np

s3_client = boto3.client('s3')
#object_categories = ['ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'basketball-hoop', 'bat', 'bathtub', 'bear', 'beer-mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai-101', 'boom-box', 'bowling-ball', 'bowling-pin', 'boxing-glove', 'brain-101', 'breadmaker', 'buddha-101', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car-tire', 'cartman', 'cd', 'centipede', 'cereal-box', 'chandelier-101', 'chess-board', 'chimp', 'chopsticks', 'cockroach', 'coffee-mug', 'coffin', 'coin', 'comet', 'computer-keyboard', 'computer-monitor', 'computer-mouse', 'conch', 'cormorant', 'covered-wagon', 'cowboy-hat', 'crab-101', 'desk-globe', 'diamond-ring', 'dice', 'dog', 'dolphin-101', 'doorknob', 'drinking-straw', 'duck', 'dumb-bell', 'eiffel-tower', 'electric-guitar-101', 'elephant-101', 'elk', 'ewer-101', 'eyeglasses', 'fern', 'fighter-jet', 'fire-extinguisher', 'fire-hydrant', 'fire-truck', 'fireworks', 'flashlight', 'floppy-disk', 'football-helmet', 'french-horn', 'fried-egg', 'frisbee', 'frog', 'frying-pan', 'galaxy', 'gas-pump', 'giraffe', 'goat', 'golden-gate-bridge', 'goldfish', 'golf-ball', 'goose', 'gorilla', 'grand-piano-101', 'grapes', 'grasshopper', 'guitar-pick', 'hamburger', 'hammock', 'harmonica', 'harp', 'harpsichord', 'hawksbill-101', 'head-phones', 'helicopter-101', 'hibiscus', 'homer-simpson', 'horse', 'horseshoe-crab', 'hot-air-balloon', 'hot-dog', 'hot-tub', 'hourglass', 'house-fly', 'human-skeleton', 'hummingbird', 'ibis-101', 'ice-cream-cone', 'iguana', 'ipod', 'iris', 'jesus-christ', 'joy-stick', 'kangaroo-101', 'kayak', 'ketch-101', 'killer-whale', 'knife', 'ladder', 'laptop-101', 'lathe', 'leopards-101', 'license-plate', 'lightbulb', 'light-house', 'lightning', 'llama-101', 'mailbox', 'mandolin', 'mars', 'mattress', 'megaphone', 'menorah-101', 'microscope', 'microwave', 'minaret', 'minotaur', 'motorbikes-101', 'mountain-bike', 'mushroom', 'mussels', 'necktie', 'octopus', 'ostrich', 'owl', 'palm-pilot', 'palm-tree', 'paperclip', 'paper-shredder', 'pci-card', 'penguin', 'people', 'pez-dispenser', 'photocopier', 'picnic-table', 'playing-card', 'porcupine', 'pram', 'praying-mantis', 'pyramid', 'raccoon', 'radio-telescope', 'rainbow', 'refrigerator', 'revolver-101', 'rifle', 'rotary-phone', 'roulette-wheel', 'saddle', 'saturn', 'school-bus', 'scorpion-101', 'screwdriver', 'segway', 'self-propelled-lawn-mower', 'sextant', 'sheet-music', 'skateboard', 'skunk', 'skyscraper', 'smokestack', 'snail', 'snake', 'sneaker', 'snowmobile', 'soccer-ball', 'socks', 'soda-can', 'spaghetti', 'speed-boat', 'spider', 'spoon', 'stained-glass', 'starfish-101', 'steering-wheel', 'stirrups', 'sunflower-101', 'superman', 'sushi', 'swan', 'swiss-army-knife', 'sword', 'syringe', 'tambourine', 'teapot', 'teddy-bear', 'teepee', 'telephone-box', 'tennis-ball', 'tennis-court', 'tennis-racket', 'theodolite', 'toaster', 'tomato', 'tombstone', 'top-hat', 'touring-bike', 'tower-pisa', 'traffic-light', 'treadmill', 'triceratops', 'tricycle', 'trilobite-101', 'tripod', 't-shirt', 'tuning-fork', 'tweezer', 'umbrella-101', 'unicorn', 'vcr', 'video-projector', 'washing-machine', 'watch-101', 'waterfall', 'watermelon', 'welding-mask', 'wheelbarrow', 'windmill', 'wine-bottle', 'xylophone', 'yarmulke', 'yo-yo', 'zebra', 'airplanes-101', 'car-side-101', 'faces-easy-101', 'greyhound', 'tennis-shoes', 'toad', 'clutter']
object_categories = ['NO_SHIP', 'SHIP']

def list_objects(s3_client, bucket, prefix):
    response = s3_client.list_objects(Bucket=bucket, Prefix=prefix)
    objects = [content['Key'] for content in response['Contents']]
    return objects

def get_label(s3_client, bucket, prefix):
    filename = prefix.split('/')[-1]
    s3_client.download_file(bucket, prefix, filename)
    with open(filename) as f:
        data = json.load(f)
        index = np.argmax(data['prediction'])
        probability = data['prediction'][index]
    print("Result: label - " + object_categories[index] + ", probability - " + str(probability))
    return object_categories[index], probability

inputs = list_objects(s3_client, bucket, urlparse(batch_input).path.lstrip('/'))
print("Sample inputs: " + str(inputs[:2]))

outputs = list_objects(s3_client, bucket, batch_job_name + "/output")
print("Sample output: " + str(outputs[:2]))

# Check prediction result of the first 2 images
[get_label(s3_client, bucket, prefix) for prefix in outputs[0:10]]

Sample inputs: ['tiny3/test/SHIP/16a177779.jpg', 'tiny3/test/SHIP/6d69b8354.jpg']
Sample output: ['ships-model-ship-validation-2019-02-13-21-57-21/output/16a177779.jpg.out', 'ships-model-ship-validation-2019-02-13-21-57-21/output/6d69b8354.jpg.out']
Result: label - NO_SHIP, probability - 0.9800084829330444
Result: label - NO_SHIP, probability - 0.650273859500885
Result: label - NO_SHIP, probability - 0.8613647222518921


[('NO_SHIP', 0.9800084829330444),
 ('NO_SHIP', 0.650273859500885),
 ('NO_SHIP', 0.8613647222518921)]

### Realtime inference

We now host the model with an endpoint and perform realtime inference.

This section involves several steps,
1. [Create endpoint configuration](#CreateEndpointConfiguration) - Create a configuration defining an endpoint.
1. [Create endpoint](#CreateEndpoint) - Use the configuration to create an inference endpoint.
1. [Perform inference](#PerformInference) - Perform inference on some input data using the endpoint.
1. [Clean up](#CleanUp) - Delete the endpoint and model

#### Create Endpoint Configuration
At launch, we will support configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way.

In addition, the endpoint configuration describes the instance type required for model deployment, and at launch will describe the autoscaling configuration.

In [ ]:
from time import gmtime, strftime

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

#### Create Endpoint
Lastly, the customer creates the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [ ]:
%%time
import time

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = job_name_prefix + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sagemaker.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

Finally, now the endpoint can be created. It may take sometime to create the endpoint...

In [ ]:
# get the status of the endpoint
response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))


# wait until the status has changed
sagemaker.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)


# print the status of the endpoint
endpoint_response = sagemaker.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

If you see the message,

> `Endpoint creation ended with EndpointStatus = InService`

then congratulations! You now have a functioning inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.

We will finally create a runtime object from which we can invoke the endpoint.

#### Perform Inference
Finally, the customer can now validate the model for use. They can obtain the endpoint from the client library using the result from previous operations, and generate classifications from the trained model using that endpoint.


In [ ]:
import boto3
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

##### Download test image

In [ ]:
!wget -O /tmp/test.jpg http://www.vision.caltech.edu/Image_Datasets/Caltech256/images/008.bathtub/008_0007.jpg
file_name = '/tmp/test.jpg'
# test image
from IPython.display import Image
Image(file_name)  

In [ ]:
import json
import numpy as np
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)
result = response['Body'].read()
# result will be in json format and convert it to ndarray
result = json.loads(result)
# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index
index = np.argmax(result)
object_categories = ['ak47', 'american-flag', 'backpack', 'baseball-bat', 'baseball-glove', 'basketball-hoop', 'bat', 'bathtub', 'bear', 'beer-mug', 'billiards', 'binoculars', 'birdbath', 'blimp', 'bonsai-101', 'boom-box', 'bowling-ball', 'bowling-pin', 'boxing-glove', 'brain-101', 'breadmaker', 'buddha-101', 'bulldozer', 'butterfly', 'cactus', 'cake', 'calculator', 'camel', 'cannon', 'canoe', 'car-tire', 'cartman', 'cd', 'centipede', 'cereal-box', 'chandelier-101', 'chess-board', 'chimp', 'chopsticks', 'cockroach', 'coffee-mug', 'coffin', 'coin', 'comet', 'computer-keyboard', 'computer-monitor', 'computer-mouse', 'conch', 'cormorant', 'covered-wagon', 'cowboy-hat', 'crab-101', 'desk-globe', 'diamond-ring', 'dice', 'dog', 'dolphin-101', 'doorknob', 'drinking-straw', 'duck', 'dumb-bell', 'eiffel-tower', 'electric-guitar-101', 'elephant-101', 'elk', 'ewer-101', 'eyeglasses', 'fern', 'fighter-jet', 'fire-extinguisher', 'fire-hydrant', 'fire-truck', 'fireworks', 'flashlight', 'floppy-disk', 'football-helmet', 'french-horn', 'fried-egg', 'frisbee', 'frog', 'frying-pan', 'galaxy', 'gas-pump', 'giraffe', 'goat', 'golden-gate-bridge', 'goldfish', 'golf-ball', 'goose', 'gorilla', 'grand-piano-101', 'grapes', 'grasshopper', 'guitar-pick', 'hamburger', 'hammock', 'harmonica', 'harp', 'harpsichord', 'hawksbill-101', 'head-phones', 'helicopter-101', 'hibiscus', 'homer-simpson', 'horse', 'horseshoe-crab', 'hot-air-balloon', 'hot-dog', 'hot-tub', 'hourglass', 'house-fly', 'human-skeleton', 'hummingbird', 'ibis-101', 'ice-cream-cone', 'iguana', 'ipod', 'iris', 'jesus-christ', 'joy-stick', 'kangaroo-101', 'kayak', 'ketch-101', 'killer-whale', 'knife', 'ladder', 'laptop-101', 'lathe', 'leopards-101', 'license-plate', 'lightbulb', 'light-house', 'lightning', 'llama-101', 'mailbox', 'mandolin', 'mars', 'mattress', 'megaphone', 'menorah-101', 'microscope', 'microwave', 'minaret', 'minotaur', 'motorbikes-101', 'mountain-bike', 'mushroom', 'mussels', 'necktie', 'octopus', 'ostrich', 'owl', 'palm-pilot', 'palm-tree', 'paperclip', 'paper-shredder', 'pci-card', 'penguin', 'people', 'pez-dispenser', 'photocopier', 'picnic-table', 'playing-card', 'porcupine', 'pram', 'praying-mantis', 'pyramid', 'raccoon', 'radio-telescope', 'rainbow', 'refrigerator', 'revolver-101', 'rifle', 'rotary-phone', 'roulette-wheel', 'saddle', 'saturn', 'school-bus', 'scorpion-101', 'screwdriver', 'segway', 'self-propelled-lawn-mower', 'sextant', 'sheet-music', 'skateboard', 'skunk', 'skyscraper', 'smokestack', 'snail', 'snake', 'sneaker', 'snowmobile', 'soccer-ball', 'socks', 'soda-can', 'spaghetti', 'speed-boat', 'spider', 'spoon', 'stained-glass', 'starfish-101', 'steering-wheel', 'stirrups', 'sunflower-101', 'superman', 'sushi', 'swan', 'swiss-army-knife', 'sword', 'syringe', 'tambourine', 'teapot', 'teddy-bear', 'teepee', 'telephone-box', 'tennis-ball', 'tennis-court', 'tennis-racket', 'theodolite', 'toaster', 'tomato', 'tombstone', 'top-hat', 'touring-bike', 'tower-pisa', 'traffic-light', 'treadmill', 'triceratops', 'tricycle', 'trilobite-101', 'tripod', 't-shirt', 'tuning-fork', 'tweezer', 'umbrella-101', 'unicorn', 'vcr', 'video-projector', 'washing-machine', 'watch-101', 'waterfall', 'watermelon', 'welding-mask', 'wheelbarrow', 'windmill', 'wine-bottle', 'xylophone', 'yarmulke', 'yo-yo', 'zebra', 'airplanes-101', 'car-side-101', 'faces-easy-101', 'greyhound', 'tennis-shoes', 'toad', 'clutter']
print("Result: label - " + object_categories[index] + ", probability - " + str(result[index]))

#### Clean up

When we're done with the endpoint, we can just delete it and the backing instances will be released.  Run the following cell to delete the endpoint.

In [ ]:
sage.delete_endpoint(EndpointName=endpoint_name)